In [32]:
#pip install downcast

In [1]:
import pandas as pd
import pickle
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from downcast import reduce
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [2]:
calendar_=pd.read_csv(r'calendar.csv')
sales_train_evaluation_=pd.read_csv(r'sales_train_evaluation.csv')
sell_prices_=pd.read_csv(r'sell_prices.csv')

* Replace 'NaN' values by 'no_event' in event columns

In [3]:
cat=['event_name_1','event_type_1','event_name_2','event_type_2']
for i in cat:
    calendar_[i].fillna('no_event',inplace=True)

* Adding feature 'is_weekend' which tells about that day is weekend or not

In [4]:
f=lambda x: 1 if x<=2 else 0
calendar_['is_weekend']=calendar_['wday'].map(f) 
calendar_['is_weekend']=calendar_['is_weekend'].astype(np.int8)

* Adding feature 'month_day' which tells day of the month

In [5]:
m=calendar_["date"].tolist()
m=[i.split("-")[2] for i in m]
calendar_["month_day"]=m
calendar_['month_day']=calendar_['month_day'].astype(np.int8)

* Adding feature 'month_week_number' which tells which week of the month

In [6]:
calendar_['month_week_number']=(calendar_['month_day']-1) // 7 + 1 
calendar_['month_week_number']=calendar_['month_week_number'].astype(np.int8)

* Adding feature 'events_per_day' which tells us number of events on particular day

In [7]:
f=lambda x: 0 if x=='no_event' else 1
calendar_['events_per_day']=calendar_['event_type_1'].map(f) 
index=calendar_.index 
indices=index[calendar_['event_type_2']!='no_event'].tolist()
for i in indices:
    calendar_['events_per_day'][i]+=1
calendar_['events_per_day']=calendar_['events_per_day'].astype(np.int8)

In [8]:
calendar_=reduce(calendar_)

In [9]:
calendar_.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day
0,2011-01-29,11101,Saturday,1,1,2011,d_1,no_event,no_event,no_event,no_event,0,0,0,1,29,5,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,no_event,no_event,no_event,no_event,0,0,0,1,30,5,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,no_event,no_event,no_event,no_event,0,0,0,0,31,5,0


In [10]:
sales_train_evaluation_=reduce(sales_train_evaluation_)

In [11]:
sell_prices_=reduce(sell_prices_)

In [12]:
calendar_['date'].nunique()

1969

* No of days data present is 1969

In [13]:
sales_train_evaluation_[['id','item_id','dept_id','cat_id','store_id','state_id']].nunique()

id          30490
item_id      3049
dept_id         7
cat_id          3
store_id       10
state_id        3
dtype: int64

* No. of unique:
  
  * id: 30490

  * items: 3049

  * Departments: 7

  * Category: 3

  * Stores: 10

  * States: 3


## Melting

* To make analysis of data in table easier we can reshape the data into a more computer-friendly form using pandas in Python. pandas.melt() is one of the function to do so

* pandas.melt() unpivots a DataFrame from wide format to long format.

* melt() function is useful to massage a DataFrame into a format where one or more columns are identifier variables while all other columns considered measured variables are unpivoted to the row axis leaving just two non-identifier columns variable and value.

In [14]:
sales=pd.melt(sales_train_evaluation_,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='demand')
sales=pd.merge(sales,calendar_,on='d',how='left')
sales=pd.merge(sales,sell_prices_,on=['item_id','store_id','wm_yr_wk'],how='left')

In [15]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,no_event,0,0,0,1,29,5,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,no_event,0,0,0,1,29,5,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,no_event,0,0,0,1,29,5,0,NaN


In [16]:
l=[]
for i in sales['d']:
    l.append(i.split('_')[1])
sales['day']=l
sales['day']=sales['day'].astype(np.int16)

In [17]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,day
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,0,0,0,1,29,5,0,NaN,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,0,0,0,1,29,5,0,NaN,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,0,0,0,1,29,5,0,NaN,1


* Since we got many rows with 'NaN' values in Sell Price 

* Thus replacing 'NaN" in 'sell_price' feature with the mean value 

In [18]:
sales['sell_price']=sales['sell_price'].fillna(sales.groupby('id')['sell_price'].transform('mean'))

In [19]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,event_type_2,snap_CA,snap_TX,snap_WI,is_weekend,month_day,month_week_number,events_per_day,sell_price,day
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,0,0,0,1,29,5,0,8.281250,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,0,0,0,1,29,5,0,3.970703,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,no_event,0,0,0,1,29,5,0,2.970703,1


In [20]:
sales=reduce(sales)

## Lags

* Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

* Lag is expressed in a time unit & corresponds to the amount of data history we allow the model to use when making the prediction.

* Here we have applied Lags on 'demand' column.

* The maximum Lags taken is 70 days

In [21]:
lags=[28,35,42,49,56,63,70]
for i in tqdm(lags):
    sales['lag_'+str(i)]=sales.groupby(['id'])['demand'].shift(i)

100%|██████████| 7/7 [03:21<00:00, 28.84s/it]


In [22]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,events_per_day,sell_price,day,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0,8.281250,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0,3.970703,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0,2.970703,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Replacing 'NaN' in 'lags' features with 0

In [23]:
lags=['lag_28','lag_35','lag_42','lag_49','lag_56','lag_63','lag_70']
for i in lags:
    sales[i]=sales[i].fillna(0) 

In [24]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,events_per_day,sell_price,day,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0,8.281250,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0,3.970703,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0,2.970703,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
sales=reduce(sales)

## Rolling-Median

* Rolling is a very useful operation for time series data.

* Rolling means creating a rolling window with a specified size & perform calculations on data in this window which of course rolls through data

* Here we have computing Rolling-Median on 'demand' column.

* The maximum Window size taken is 42

In [26]:
window=[7,14,28,35,42]
for i in tqdm(window):
    sales['rolling_median_'+str(i)]=sales.groupby(['id'])['demand'].transform(lambda s: s.rolling(i,center=False).median())

100%|██████████| 5/5 [24:59<00:00, 299.97s/it]


* Replacing 'NaN' in 'rolling_ median' features with 0

In [27]:
window=['rolling_median_7','rolling_median_14','rolling_median_28','rolling_median_35','rolling_median_42']
for i in window:
    sales[i]=sales[i].fillna(0) 

In [28]:
sales=reduce(sales)

In [29]:
sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_median_7,rolling_median_14,rolling_median_28,rolling_median_35,rolling_median_42
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Label-Encoding

* Encoding refers to converting the labels into numeric form so as to convert it into the machine-readable form.

* Machine learning algorithms can then decide in a better way on how those labels must be operated.

* It is an important pre-processing step for the structured dataset in supervised learning

In [30]:
labelencoder=LabelEncoder() 
category=['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id']
for i in tqdm(category):
    sales[i+'_']=labelencoder.fit_transform(sales[i])

100%|██████████| 10/10 [03:03<00:00, 18.40s/it]


In [31]:
sales=reduce(sales)

* Drop all the categorical columns bcoz we already added coresponding columns with label-encoding 

In [32]:
sales=sales.drop(['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

In [33]:
sales.head(3)

,d,demand,date,wm_yr_wk,weekday,wday,month,year,snap_CA,snap_TX,...,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_
0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,30,4,4,2,14370,1437,3,1,0,0
1,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,30,4,4,2,14380,1438,3,1,0,0
2,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,...,30,4,4,2,14390,1439,3,1,0,0


In [34]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 39 columns):
 #   Column             Dtype         
---  ------             -----         
 0   d                  category      
 1   demand             int16         
 2   date               datetime64[ns]
 3   wm_yr_wk           int16         
 4   weekday            category      
 5   wday               int8          
 6   month              int8          
 7   year               int16         
 8   snap_CA            int8          
 9   snap_TX            int8          
 10  snap_WI            int8          
 11  is_weekend         int8          
 12  month_day          int8          
 13  month_week_number  int8          
 14  events_per_day     int8          
 15  sell_price         float16       
 16  day                int16         
 17  lag_28             float16       
 18  lag_35             float16       
 19  lag_42             float16       
 20  lag_49             flo

In [35]:
sales=sales.drop(['d','date','weekday'],axis=1)

In [36]:
sales.head(3)

,demand,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,is_weekend,month_day,...,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_
0,0,11101,1,1,2011,0,0,0,1,29,...,30,4,4,2,14370,1437,3,1,0,0
1,0,11101,1,1,2011,0,0,0,1,29,...,30,4,4,2,14380,1438,3,1,0,0
2,0,11101,1,1,2011,0,0,0,1,29,...,30,4,4,2,14390,1439,3,1,0,0


In [39]:
sales.to_pickle('data_final.pkl')